In [25]:
! pip install -qU langchain-qdrant

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

c:\Users\thaia\Desktop\Financial Chatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
import json

paths = [r"C:\Users\thaia\Desktop\Financial Chatbot\data\yuanta_chung_khoan_20251226_173528.json",\
    r'C:\Users\thaia\Desktop\Financial Chatbot\data\yuanta_chung_khoan_20251226_173647.json']

data = []
for path in paths:
	with open(path, "r", encoding="utf-8") as f:
		data += json.load(f)


In [16]:
from langchain_core.documents import Document

docs = []

for item in data:
    # Bạn chọn field muốn cho vào page_content
    text = f"{item.get('title', '')}\n\n{item.get('content', '')}"

    doc = Document(
        page_content=text,
        metadata={
            "title": item.get("title"),
            "content": item.get("content"),
        }
    )
    docs.append(doc)

print("Số document:", len(docs))

Số document: 58


In [3]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams


# url = "http://localhost:6333/"

# qdrant = QdrantVectorStore.from_documents(
#     docs,
#     embeddings,
#     url=url,
#     prefer_grpc=True,
#     collection_name="simple_stock",
# )

In [9]:
qdrant = QdrantVectorStore.from_existing_collection(
    embedding=embeddings,
    collection_name="simple_stock",
    url="http://localhost:6333",
)

In [14]:
results = qdrant.similarity_search_with_score(
    "Cổ đông là gì",
    k=58
)
scores = [score for _, score in results]
mean_score = sum(scores) / len(scores)
var = sum((score - mean_score) ** 2 for score in scores) / len(scores)
std_dev = var ** 0.5
threshold = mean_score + std_dev
print(f"Mean score: {mean_score}, Standard Deviation: {std_dev}")
for res, score in results:
    if score >= threshold:
    	print(f"* {score} [{res.metadata}]")

Mean score: 0.4818047331034483, Standard Deviation: 0.04712805291009489
* 0.60364133 [{'title': '#24 Vị thế là gì? Ký quỹ là gì?', 'content': ['Xin chào các bạn đang đến với Ysedu chuỗi videos đào tạo kiến thức đầu tư của công ty chứng khoán Yuanta Việt Nam.Và hôm nay chúng ta sẽ cùng tiềm hiểu về Vị thế và Ký quỹ.', 'Vị thế', 'Vị thế một chứng khoán phái sinh (CKPS) tại một thời điểm là trạng thái giao dịch và khối lượng chứng khoán phái sinh còn hiệu lực mà NĐT đang nắm giữ tính đến thời điểm đó.', 'Trong giao dịch hợp đồng tương lai (HĐTL), vị thế một CKPS bao gồm vị thế mua và vị thế bán.', 'Vị thế mua: Khi NĐT kỳ vọng giá của tài sản cơ sở (ví dụ chỉ số VN30 Index) sẽ tăng trong tương lai, nhà đầu tư sẽ mua hợp đồng, được gọi là mở vị thế mua hay tham gia vị thế mua (bên mua). Để đóng vị thế NĐT phải bán hợp đồng đi hoặc nắm giữ vị thế đến khi đáo hạn.', 'Vị thế bán: Khi NĐT là người đang nắm giữ tài sản cơ sở, cho rằng giá của tài sản cơ sở sẽ giảm trong tương lai, sẽ bán hợp đồn

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

all_docs = [doc.page_content for doc in docs]
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(all_docs)

In [ ]:
import joblib

# Lưu vectorizer
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")

# Lưu ma trận TF-IDF
joblib.dump(tfidf_matrix, "tfidf_matrix.pkl")

['tfidf_matrix.pkl']

In [27]:
import numpy as np

def rag_search(query):
    # Step 1: vector search trong Qdrant
    results = qdrant.similarity_search_with_score(query, k=58)

    # Step 2: thống kê để tính threshold
    scores = [score for (_, score) in results]
    mean_score = np.mean(scores)
    std_dev = np.std(scores)
    threshold = mean_score + std_dev

    print(f"Mean score: {mean_score}, Std: {std_dev}")

    # Step 3: TF-IDF score cho query
    q_sparse = vectorizer.transform([query])

    hybrid_results = []

    for doc, dense_score in results:
        # TF-IDF của chunk
        doc_sparse = vectorizer.transform([doc.page_content])
        tfidf_score = (q_sparse @ doc_sparse.T).toarray()[0][0]

        # Hybrid score: dense 70%, sparse 30% (tùy chỉnh)
        hybrid_score = 0.7 * dense_score + 0.3 * tfidf_score

        # Chỉ giữ nếu qua ngưỡng thống kê
        if dense_score >= threshold:
            hybrid_results.append((doc, hybrid_score))

    # Step 4: Sắp xếp lại theo hybrid score
    hybrid_results.sort(key=lambda x: x[1], reverse=True)

    return hybrid_results[:5]
